# Crab Nebula example for low zenith range

In [ ]:
from datetime import datetime, timezone
from pathlib import Path


from astroplan import Observer, FixedTarget
from astropy.time import Time
import astropy.units as u
from astropy.visualization import quantity_support
import matplotlib.pyplot as plt
from ipywidgets import interact, DatetimePicker

from iact_estimator import RESOURCES_PATH
from iact_estimator.io import read_yaml
from iact_estimator.core import (
    initialize_model,
    prepare_data,
    source_detection,
    calculate,
)
from iact_estimator.plots.physics import plot_spectrum, plot_sed
from iact_estimator.plots.observability import plot_transit, plot_altitude_airmass
from iact_estimator.plots.wobble_skymap import plot_skymap_with_wobbles, load_wobbles

In [ ]:
output_path = Path.cwd()
config = read_yaml(RESOURCES_PATH / "config.yml")
source_name = "Crab"

observer = Observer.at_site("Roque de los Muchachos")

crab = FixedTarget.from_name(source_name)

In [ ]:
plotting_options = config["plotting_options"]
use_seaborn = config["use_seaborn"]
if use_seaborn:
    import seaborn as sns

    seaborn_options = config["seaborn_options"]
    sns.set_theme(**seaborn_options)


assumed_spectrum = initialize_model(config)

plot_energy_bounds = [
    u.Quantity(plotting_options["min_energy"]),
    u.Quantity(plotting_options["max_energy"]),
]

## Source transit

In [ ]:
target_source = FixedTarget.from_name(source_name)
observer = Observer.at_site("Roque de los Muchachos")

date_time = DatetimePicker(
    value=datetime.now(timezone.utc), description="Select a datetime", disabled=False
)

crab = FixedTarget.from_name("Crab")
plot_crab = True if (crab.coord == target_source.coord) else False


def interactive_plot_transit(date_time):
    with quantity_support():
        plot_transit(
            config,
            source_name,
            target_source,
            observer,
            time=Time(date_time).utc,
            merge_profiles=True,
            plot_crab=False,
            savefig=False,
        )


interact(interactive_plot_transit, date_time=date_time)
plt.show()

## Altitude and airmass

In [ ]:
date_time = DatetimePicker(
    value=datetime.now(timezone.utc), description="Select a datetime", disabled=False
)


def plot_alt(date_time):
    print(date_time)

    plot_altitude_airmass(
        config,
        source_name,
        target_source,
        observer,
        time=Time(date_time).utc,
        brightness_shading=True,
        airmass_yaxis=True,
        savefig=False,
    )


interact(plot_alt, date_time=date_time)
plt.show()

## Spectrum

In [ ]:
plot_spectrum(
    config,
    plot_energy_bounds,
    assumed_spectrum,
    source_name,
    plotting_options,
    savefig=False,
)

## Spectral energy distribution

In [ ]:
energy_bins, gamma_rate, background_rate = prepare_data(config)

en, sed, dsed, sigmas, detected = calculate(
    energy_bins, gamma_rate, background_rate, config, assumed_spectrum
)

combined_significance = source_detection(sigmas, u.Quantity(config["observation_time"]))

In [ ]:
annotation_options = {"rotation": 45, "xytext": (10, 10), "size": 15}

with quantity_support():
    plot_sed(
        config,
        sigmas,
        combined_significance,
        source_name,
        assumed_spectrum,
        en,
        sed,
        dsed,
        detected,
        savefig=False,
        annotation_options=annotation_options,
    )
    plt.ylim(1.0e-12, 2.0e-10)

In [ ]:
instrument_fov = u.Quantity(config["fov"])
wobble_offsets, wobble_angles = load_wobbles(config["wobbles"])
plot_skymap_with_wobbles(
    target_source,
    observer,
    instrument_fov,
    wobble_angles,
    wobble_offsets,
    config,
)